In [1]:
%%configure -f
{"driverMemory": "8G", 
 "executorMemory": "16G", 
 "executorCores": 8, 
 "numExecutors": 8,
 "name": "pc",
 "conf": {"spark.dynamicAllocation.enabled":"false",
          "spark.pyspark.python": "python3",
          "spark.pyspark.virtualenv.enabled": "true",
          "spark.pyspark.virtualenv.type":"native",
          "spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
         }
}

In [2]:
# "spark.pyspark.virtualenv.enabled":"true",
# "spark.pyspark.virtualenv.type":"native"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
135,application_1627924993397_0059,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
spark

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
sc.install_pypi_package("pandas==0.25.1") #Install pandas version 0.25.1 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/7e/ab/ea76361f9d3e732e114adcd801d2820d5319c23d0ac5482fa3b412db217e/pandas-0.25.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/36/7a/87837f39d0296e723bb9b62bbb257d0355c7f6128853c78955f57342a56d/python_dateutil-2.8.2-py2.py3-none-any.whl

In [5]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import ArrayType, StringType, LongType, IntegerType, DoubleType, StructType, StructField

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
sc.install_pypi_package("scipy")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/b5/6b/8bc0b61ebf824f8c3979a31368bbe38dd247590049a994ab0ed077cb56dc/scipy-1.7.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl

In [7]:
#sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from scipy.stats import entropy
import pandas as pd
from operator import itemgetter

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# configs

In [9]:
top_k=1
model_name='raw'  ## dampened, boosting, raw

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
trend_score_col="category_trend_scores",
probability_col="probability",
category_col="category_id",
model_date_col="model_date"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
model_date="2021-07-27"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# input paths

In [35]:
model_path = f"s3a://pai-gm-midgar-production-ml-jobs/shinra/modelling/weighted_trendfiltering_banner_events/user_categories_raw/model_date={model_date}"
model_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3a://pai-gm-midgar-production-ml-jobs/shinra/modelling/weighted_trendfiltering_banner_events/user_categories_raw/model_date=2021-07-27'

In [36]:
latest_cma_mapping_file = "s3://midgar-aws-workspace/prod/shinra/base/l1_cma_name_to_id_map/2021-07-31"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# output paths

In [37]:
topk_path = f"s3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/topk_metrics/model={model_name}/"

topk_dist_path = f"{topk_path}/dt={model_date}"
topk_dist_path

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/topk_metrics/model=raw//dt=2021-07-27'

# work

In [15]:
def get_map_table(spark, latest_cma_mapping_file, c0, c1):
    cma_mapping = (spark.read.option("header", "false")
                   .csv(latest_cma_mapping_file.replace("s3", "s3a"))
                   .withColumnRenamed("_c0", c0)
                   .withColumnRenamed("_c1", c1))
    return cma_mapping

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
cma_mapping = get_map_table(spark, latest_cma_mapping_file, "category_name","top_cat")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
cma_mapping.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

123

In [18]:
def model_servable_categories(spark):
    model_categories_path = "s3://midgar-aws-workspace/prod/shinra/telescope/dashboard/MODEL_SERVABLE_CATEGORIES"
    model_categories = (spark
                        .read
                        .option("header", True)
                        .csv(model_categories_path)
                        .agg(F.collect_set(F.col("cma_id")).alias("categories"))).collect()[0]["categories"]
    return model_categories

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
model_categories = model_servable_categories(spark)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
len(model_categories)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

40

In [21]:
spark.read.parquet(model_path).select(F.col('category_trend_scores.probability')).show(1,False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|probability                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
+---------------------------------------------------------------------------------

In [22]:
def rank_model_predictions(top_k):
    arrays_zip_ = F.udf(lambda x, y: list(zip(x, y)),
                        ArrayType(StructType([
                            # Choose Datatype according to requirement
                            StructField('0', DoubleType()),
                            StructField('1', StringType())
                        ])))
    print(f'model_path: {model_path}')
    model_df = spark.read.parquet(model_path)
    ranked_model_output = (model_df.select("customer_id", "category_trend_scores")
                           .withColumn("sorted_prediction",
                                       F.sort_array(
                                           arrays_zip_(F.col('category_trend_scores.probability'),
                                                       F.col('category_trend_scores.category_id')),
                                           asc=False))
                           .withColumn("predicted_top_category",
                                       F.array([F.col("sorted_prediction.1")[i] for i in range(top_k)]))
                           .select("customer_id", "predicted_top_category")
                           )
    return ranked_model_output

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
def unique_categories_in_top_k(ranked_model_output, ranked_category_array_col="predicted_top_category"):
    return (ranked_model_output
            .select(ranked_category_array_col)
            .select(F.posexplode(ranked_category_array_col))
            .withColumnRenamed("col", "top_cat")
            .withColumn("rank", F.col("pos") + 1)
            .groupBy("top_cat", "rank")
            .agg(F.count("top_cat").alias("total_model_rec_cnt")))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
def cumulative_model_output_analysis(aggregated_model_ranked_output, topk_dist_path):
    window_by_category_and_rank = (Window.partitionBy('top_cat').orderBy('rank')
                                   .rangeBetween(Window.unboundedPreceding, 0))
    rank_window_by_rank = Window.partitionBy('rank')
    category_cumulative_df = (
        aggregated_model_ranked_output.withColumn('cum_sum',
                                                  F.sum("total_model_rec_cnt").over(window_by_category_and_rank))
            .withColumn("percentage", F.col("cum_sum") / F.sum('cum_sum').over(rank_window_by_rank))
            .withColumnRenamed("cum_sum", "category_rec_count")
            .withColumnRenamed("rank", "k")
            .select("k", "top_cat", "category_name", "category_rec_count", "percentage"))

    print(topk_dist_path)
    category_cumulative_df.coalesce(1).write.mode("overwrite").parquet(topk_dist_path)

    return spark.read.parquet(topk_dist_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
def coverage_calculation(category_cumulative_df, k=1):
    model_predicted_categories = (category_cumulative_df
                                  .filter(F.col("k") <= k)
                                  .select("top_cat")
                                  .distinct()
                                  .toPandas()["top_cat"].to_list()
                                 )

    return len(set(model_predicted_categories) & set(model_categories)) / len(model_categories)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
def entropy_calculation(category_cumulative_df, top_k=1) -> float:
    top_k_entropy = {}
    for i in range(1, top_k + 1):
        df_k = category_cumulative_df[category_cumulative_df["k"] == i]
        top_k_entropy[str(i)] = entropy(df_k["percentage"])
    top_k_entropy["dt"] = model_date
    df = pd.DataFrame(top_k_entropy, index=[0])
    for i in df.columns:
        df[i] = df[i].astype(str)
    print(f"entropy for top1 categories is {top_k_entropy['1']}")
    return float(top_k_entropy['1'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
def gini_index(category_cumulative_df, top_k=1):
    gini_base_table = category_cumulative_df.filter(F.col("k") == top_k) \
        .select("top_cat", "percentage") \
        .toPandas()
    p = pd.Series(gini_base_table.percentage.values, index=gini_base_table.top_cat).to_dict()
    j = 1
    n = len(p)
    G = 0
    for item, weight in sorted(p.items(), key=itemgetter(1)):
        G += (2 * j - n - 1) * weight
        j += 1
    print("gini index is " + str(G / float(n - 1)))
    return G / float(n - 1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
reranked_model = rank_model_predictions(top_k=top_k)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model_path: s3a://pai-gm-midgar-production-ml-jobs/shinra/modelling/weighted_trendfiltering_banner_events/user_categories_raw/model_date=2021-07-26

In [29]:
unique_categories_in_top_k_df = unique_categories_in_top_k(reranked_model).join(cma_mapping, on="top_cat", how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
topk_dist_path


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

's3://pai-gm-midgar-shared-ml-team/pengcheng/eval/model_eval/30_day_trends/topk_metrics/model=raw//dt=2021-07-27'

In [38]:
#analyzing_df = cumulative_model_output_analysis(aggregated_model_ranked_output=unique_categories_in_top_k_df,topk_dist_path=topk_dist_path).cache()
analyzing_df=spark.read.parquet(topk_dist_path)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
coverage_num = coverage_calculation(analyzing_df, k=1)
coverage_num

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.975

In [41]:
entropy_val = entropy_calculation(analyzing_df.toPandas(), top_k=1)
entropy_val

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

entropy for top1 categories is 2.3976991644518164
2.3976991644518164

In [42]:
gini_index_num = gini_index(analyzing_df, 1)
gini_index_num


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gini index is 0.8026111472274746
0.8026111472274746